In [ ]:
# 정형데이터에 가장 뛰어난 성능을 보이는 모델
# 결정트리(Decision Tree) 기반
# 앙상블 모델
# - 랜덤포레스트(Random Forest)
# - 엑스트라 트리(Extra Trees)
# - 그레디언트 부스팅(Gradient Boosting)
# - 히스토그램 기반 그레디언트 부스팅(Histogram-base Gradient Boosting)

## 1. 랜덤포레스트(Random Forest)

In [ ]:
# 앙상블 모델 중 가장 대표격 모델로 사용
# - 안정적인 성능으로 널리 사용됨
# - 앙상블 모델 중 가장 먼저 시도하는 모델
# - 훈련데이터에서 과대적합되는 것을 막아줌
# - 검증데이터와 테스트데이터에서 안정적인 성능을 얻을 수 있음

## 학습개념
# - 결정트리 하나하나를 랜덤하게 만들어서 숲을 만듦
# - 훈련데이터에서 랜덤하게 샘플을 추출하여 훈련을 완료한 후,
#   다시 원본 훈련데이터에 반환
# - 랜덤하게 추출 시 이전에 사용된 샘플을 사용할 수도 있음(중복 허용)

## 부트스트랩 샘플(Bootstrap Sample)
# - 위에 설명한 랜덤한 샘플 추출 시 중복을 허용하여 데이터를 샘플링하는 방식
# - 샘플 추출 방식
#   1) 원본에서 랜덤 샘플 추출
#   2) 훈련 이후 사용이 끝나면 원본에 반환
#   3) 다시 원본에서 샘플 추출, 이때 중복값 추출될 수 있음
#   위 순서를 반복하면서 샘플링을 통해 훈련하는 방식을 랜덤포레스트가 적용함

In [3]:
# import numpy as np
# import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
# 데이터 불러오기
file_path = '../data/08_wine.csv'
wine = pd.read_csv(file_path)

# 훈련/테스트 데이터 추출
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

# 데이터 분리
train_data, test_data, train_target, test_target = train_test_split(
    data, target, random_state=42)
    
print(train_data.shape, train_target.shape)
print(test_data.shape, test_target.shape)

(4872, 3) (4872,)
(1625, 3) (1625,)


In [5]:
# 랜덤포레스트 패키지 : sklearn.ensemble
# 랜덤포레스트 클래스(모델) : RandomForestClssifier
# 교차검증 : cross_validate()
# 교차검증 후 훈련검증결과와 테스트검증 결과 확인
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

In [6]:
# 클래스(모델)생성
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

# 교차검증
# - return_train_score=True : 검증결과 반환받기
scores = cross_validate(rf, train_data, train_target,
                        return_train_score=True, n_jobs=-1)

# 최종 훈련 평가결과, 검증결과
print(scores)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

## 과대적합이 다소 나온 편이다.

{'fit_time': array([0.24218178, 0.2288177 , 0.26561141, 0.25263715, 0.36381817]), 'score_time': array([0.05536556, 0.04576755, 0.04296255, 0.04295135, 0.02293849]), 'test_score': array([0.88      , 0.90051282, 0.90349076, 0.89014374, 0.88295688]), 'train_score': array([0.99743392, 0.99692071, 0.99846075, 0.99820421, 0.99820421])}
0.997844759088341 0.8914208392565683


In [7]:
## 훈련
rf.fit(train_data, train_target)

## 특성중요도
print(rf.feature_importances_)

[0.23155241 0.49706658 0.27138101]


In [8]:
## OOb 기능 사용
# 룬현에 참여하지 못한 잔여샘픙 사용하는 기술
# 기본은 사용 x
rf =RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(test_data,test_target)
print(rf.oob_score_)

0.8670769230769231


## 2. 엑스트라 트리(Extra Tree)

In [ ]:
# 랜덤포레스트와 유사하게 작동
# - 기본적으로 100개의 결정트리를 훈련함
# - 랜덤포레스트와의 차이점
#   : 부트스트랩 샘플링을 지원하지 않음
#   : 훈련데이터 전체를 이용하여 결정트리를 생성
#   : 무작위로 트리를 분리함
# - 사용되는 속성 : splitter = 'random'  무작위속성
# - 장점
#   : 과대적합을 막고, 검증데이터의 평가값을 높일 수 있음
#   : 특성 데이터가 많지 않은 경우에는 랜덤포레스트와 큰 차이가 없음
# - 랜덤포레스트는 불순도 등 여러가지 조건에 따라 결정트리를 생성하기 때문에 속도가 느린 반면,
#   엑스트라 트리는 랜덤하게 결정트리를 생성하기에 속도가 다소 빠르다는 장점이 있음

In [11]:
## 사용패키지 : 랜덤포레스트와 동일
# 사용되는 클래스(모델) : ExtraTreeClassifier

## 코어 전체사용, 훈련/테스트 결과값 출력, 최종 훈련/테스트 결과 확인
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_validate

In [17]:
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)

scores = cross_validate(et, train_data, train_target,
                        return_train_score=True, n_jobs=-1)

print(scores)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 0.997844759088341 0.8914208392565683 (랜덤포레스트)

{'fit_time': array([0.16778326, 0.16878033, 0.15434003, 0.18734527, 0.19133449]), 'score_time': array([0.02393579, 0.023947  , 0.026407  , 0.03597569, 0.03497267]), 'test_score': array([0.89128205, 0.89128205, 0.89938398, 0.88706366, 0.88295688]), 'train_score': array([0.99743392, 0.99692071, 0.99846075, 0.99820421, 0.99820421])}
0.997844759088341 0.8903937240035804


In [18]:
et.fit(train_data, train_target)
print(et.feature_importances_)
# [0.23155241 0.49706658 0.27138101] (랜덤포레스트)

[0.20702369 0.51313261 0.2798437 ]


In [ ]:
'''
랜덤포레스트에 비해 알코올 도수는 낮아지고 당도가 높아지니 테스트정확도가 조금 떨어진 것이 확인됨
'''

## 3. 그레디언트 부스팅(Gradient Boosting)

In [ ]:
# 길이(max_depth)가 얕은 결정트리를 사용함
#  - 기본적으로 max_depth=3 사용
#  - 결정트리는 100개 사용
## ** 기존에 다른 훈련모델의 결과가 좋지 않을 때 사용하는 모델 **
# 기존 훈련모델의 오차를 많이 보완해 줌
# 성능 향상을 위한 모델로 주로 사용됨
# 과대적합에 강하며, 일반화(과대/과소적합이 없는 상태)에 강함

# 성능향상 테스트 방법
#  - 결정트리의 갯수를 조절하면서 테스트 진행
#  - 학습률을 지원하기 때문에 학습률의 값을 증가시키면서 테스트 진행
#   : 기본 학습률은 0.1

# 단점
#  - 순서대로 트리를 추가(랜덤하지 않음)하지 않기 때문에 훈련 속도 느림
#  - 이런 느린 속도를 개선한 모델이 '히스토그램 기반 그레디언트 부스팅'

In [19]:
## 사용하는 클래스(모델) : GradientBoostingClassifier
# 객체 생성 시 아무것도 안주고 시드값만 줌
# 교차검증시에는 훈련/테스트값 결과 출력
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate

In [29]:
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_data, train_target,
                        return_train_score=True, n_jobs=-1)

print(scores)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

{'fit_time': array([0.38828325, 0.36679268, 0.37078643, 0.40415573, 0.39117765]), 'score_time': array([0.00199413, 0.00199509, 0.00303888, 0.00199556, 0.00195098]), 'test_score': array([0.86461538, 0.87794872, 0.88090349, 0.8613963 , 0.87268994]), 'train_score': array([0.89299461, 0.88555299, 0.88660852, 0.89276552, 0.88943048])}
0.8894704231708938 0.8715107671247301


In [ ]:
'''
훈련정확도가 줄어들었지만 과대적합 의심이 가장 덜한 가장 일반적인 형태이므로 선택하기 좋은 모델이다.
'''

In [30]:
gb.fit(train_data, train_target)
gb.feature_importances_

array([0.12517641, 0.73300095, 0.14182264])

In [31]:
gb.score(test_data, test_target)

0.8578461538461538

In [ ]:
'''
특성중요도를 살펴보면 나머지는 줄고 당도가 매우 증가함, 당도가 올라갈수록 성능이 좋아지는 것이 확인됨
'''

### 3-1. 학습률 적용

In [ ]:
## 학습률이 커지면 트리 보정을 강하게 하기 때문에 복잡한 모델을 만들어서 일반화 성능을 떨어뜨림
# 학습률 : (default) learning_rate=0.1 
#  - 값이 낮을수록 과대/소적합이 줄어든다, 높을수록 복잡도가 강해진다


In [26]:
gb = GradientBoostingClassifier(n_estimators=100,
                                learning_rate=0.1,
                                random_state=42)
scores = cross_validate(gb, train_data, train_target,
                        return_train_score=True, n_jobs=-1)

print(scores)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 훈련
gb.fit(train_data, train_target)

# 특성중요도
print(et.feature_importances_)

{'fit_time': array([0.35265374, 0.34857631, 0.33755088, 0.34724712, 0.33754992]), 'score_time': array([0.00205302, 0.00199437, 0.00299168, 0.00232649, 0.00304341]), 'test_score': array([0.86461538, 0.87794872, 0.88090349, 0.8613963 , 0.87268994]), 'train_score': array([0.89299461, 0.88555299, 0.88660852, 0.89276552, 0.88943048])}
0.8894704231708938 0.8715107671247301
[0.20702369 0.51313261 0.2798437 ]


## 4. 히스토그램 기반 그레디언트 부스팅(Histogram-base Gradient Boosting)

In [24]:
## 사용하는 클래스(모델) : HistGradientBoostingClassfier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_validate

In [25]:
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_data, train_target,
                        return_train_score=True, n_jobs=-1)

print(scores)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

{'fit_time': array([0.59532833, 0.53106523, 0.53505611, 0.52109456, 0.59460211]), 'score_time': array([0.00897574, 0.01043463, 0.01043558, 0.0099721 , 0.01001692]), 'test_score': array([0.87179487, 0.89333333, 0.8973306 , 0.85934292, 0.88090349]), 'train_score': array([0.93815756, 0.93482166, 0.9374038 , 0.93945613, 0.94022576])}
0.9380129799494501 0.8805410414363187


In [28]:
hgb.score(test_data, test_target)

0.8584615384615385

## 5. 사이킷런 외 다른 패키징서 지원하는 그레디언트 부스팅

### XGBoost

In [33]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate

In [36]:
xgb = XGBClassifier(tree_method='hist', random_state=42)

scores = cross_validate(xgb, train_data, train_target,
                        return_train_score=True, n_jobs=-1)

print(scores)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

{'fit_time': array([0.08877945, 0.11172414, 0.09051156, 0.0887804 , 0.11371565]), 'score_time': array([0.00398874, 0.00498176, 0.00405097, 0.00498605, 0.0039928 ]), 'test_score': array([0.87487179, 0.89333333, 0.8973306 , 0.87268994, 0.8788501 ]), 'train_score': array([0.96176546, 0.96279189, 0.95997948, 0.95690097, 0.9656234 ])}
0.9614122399872658 0.8834151529510873


### LightGBM

In [1]:
# 마이크로소프트에서 만든 히스토그램 기반 그레디언트 부스트 패키지
#  - 훈련 속도가 매우 빠르다는 장점
#  - 최신 기술을 많이 적용하고 있어 인기가 증가

# 아나콘다 : conda install -c conda-forge lightgbm 
# 파이썬 : pip install lightgbm
from lightgbm import LGBMClassifier

AttributeError: module 'snappy' has no attribute 'compress'

In [2]:
from sklearn.model_selection import cross_validate

In [6]:
lgb = LGBMClassifier(random_state=42)

scores = cross_validate(lgb, train_data, train_target,
                        return_train_score=True, n_jobs=-1)

print(scores)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

NameError: name 'LGBMClassifier' is not defined

In [ ]:
'''
xgboost보다 일반화가 적용되어 괜찮은 모델로 판단
'''